# Data Ingestion & Data Lake Construction

## Project: Beauty Lakehouse Analytics Platform

In this notebook, we implement the ingestion layer of our data architecture.

The objective of this stage is to:

- Retrieve raw CSV data from GitHub.
- Load the data into Apache Spark DataFrames.
- Validate schema and data quality.
- Perform initial exploratory analysis.

This represents the Data Lake ingestion layer in a modern big data architecture.


## Data Sources

The dataset is retrieved from a public GitHub repository:

Repository:
https://github.com/zinahalqeji/beauty_lakehouse

Raw data location:
data/raw/

Files used in this project:

- customers.csv
- products.csv
- orders.csv
- order_items.csv

These datasets represent transactional e-commerce data.

In [0]:
%pip install requests


In [0]:
import requests
import pandas as pd
from io import StringIO

base_url = "https://raw.githubusercontent.com/zinahalqeji/beauty_lakehouse/main/data/raw/"

def load_csv_from_github(filename):
    url = base_url + filename
    response = requests.get(url)
    response.raise_for_status()
    return pd.read_csv(StringIO(response.text))

# Load using pandas
customers_pd = load_csv_from_github("customers.csv")
products_pd = load_csv_from_github("products.csv")
orders_pd = load_csv_from_github("orders.csv")
order_items_pd = load_csv_from_github("order_items.csv")

# Convert to Spark DataFrames
customers_df = spark.createDataFrame(customers_pd)
products_df = spark.createDataFrame(products_pd)
orders_df = spark.createDataFrame(orders_pd)
order_items_df = spark.createDataFrame(order_items_pd)


## Tables Description

### Customers
Contains customer information such as customer_id, name, email, country.

### Products
Contains product catalog information including product_name, category, and price.

### Orders
Contains order-level information such as order_id and order_date.

### Order Items
Contains transactional details including product_id and quantity.

In [0]:
customers_df.printSchema()
products_df.printSchema()
orders_df.printSchema()
order_items_df.printSchema()

## Schema Validation

We validated the schema of each DataFrame to ensure:

- Correct column names
- Proper data types
- No structural inconsistencies

Schema validation is critical before performing joins or aggregations.

In [0]:
print("Customers:", customers_df.count())
print("Products:", products_df.count())
print("Orders:", orders_df.count())
print("Order Items:", order_items_df.count())

## Dataset Size Overview

After loading the data into Spark:

- Customers: 10,000 rows
- Products: 2,000 rows
- Orders: 100,000 rows
- Order Items: 188,573 rows

The dataset contains a significant volume of transactional data, making it suitable for distributed processing using Apache Spark.

## Data Quality Check – Missing Values

Before proceeding to transformation stages, we validate data quality.

We check for NULL values in the Customers dataset to ensure:

- Data completeness
- Reliable aggregations
- Accurate joins

Data quality validation is a critical component of big data pipelines.

In [0]:
from pyspark.sql.functions import col, sum

customers_df.select(
    [sum(col(c).isNull().cast("int")).alias(c) for c in customers_df.columns]
).display()

### Data Quality Results

The output above shows the number of NULL values for each column.

If all values are 0:
→ The dataset has no missing values in the Customers table.

This step ensures reliability before building the Document Database and Data Warehouse.

## Exploratory Analysis – Orders per Customer

We analyze how many orders each customer has placed.

This helps identify:
- Frequent buyers
- Potential high-value customers
- Purchasing behavior distribution

In [0]:
orders_df.groupBy("customer_id").count().display()

## Interpretation

The output shows the number of orders per customer.

Customers with higher order counts:
→ May represent loyal or high-value clients.

This insight will support:
- Customer Lifetime Value analysis
- Recommendation systems
- Graph-based modeling

## Exploratory Analysis – Product Distribution by Category

We examine how products are distributed across categories.

This analysis helps:
- Understand product diversity
- Identify dominant categories
- Prepare for revenue aggregation in the Data Warehouse layer

In [0]:
products_df.groupBy("category").count().display()

## Data Lake Architecture

A Data Lake stores raw data in its original format and supports large-scale distributed processing.

In this project:
- GitHub → Raw Data Storage
- Spark → Processing Layer
- Future Layers → Document DB, Warehouse, Graph Analytics

This notebook establishes the **foundation of our Lakehouse architecture**.


## Persist Data into Delta Tables (Shared Storage)

Since DBFS is disabled in our workspace, we store data in the **Hive Metastore** inside a shared database.


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS dm_project;


In [0]:
spark.catalog.setCurrentDatabase("dm_project")


In [0]:
customers_df.write.mode("overwrite").saveAsTable("customers")
products_df.write.mode("overwrite").saveAsTable("products")
orders_df.write.mode("overwrite").saveAsTable("orders")
order_items_df.write.mode("overwrite").saveAsTable("order_items")


In [0]:
spark.sql("SHOW TABLES").show()


#Summary

We successfully:
- Ingested raw CSV data from GitHub
- Converted the data into Spark DataFrames
- Validated schema and row counts
- Performed data quality checks
- Conducted exploratory analysis
- Stored the data as Delta tables in the shared `dm_project` database

This completes the **Data Lake ingestion layer** of our Lakehouse architecture.
